### Number of Trips Taken by Age and Gender

In [194]:
import csv
from csv import reader
from csv import writer
from collections import Counter
import glob
import collections
import re

pi="/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CitiBike_Data/"

people_all=[]

for file in glob.glob(pi+"*.csv"):
    path_i=file
    
    csv_=reader(open(path_i, "r"), delimiter=",")

    num=re.findall(r'\b\d+\b', file)

    people=[]

    # Skip the header
    next(csv_)

    for row in csv_:
        people.append((row[14], row[13], num[0], num[1]))        

    count=collections.Counter(people)
    
    x=count.items()
    
    people_unnested=[(a,b,c,d,e) for ((a, b, c, d),e) in x]
    
    people_all.append(people_unnested)

    with open("/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CB_real_metrics/PeopleGenderAge.csv", "w") as out:
        csv_out=csv.writer(out, delimiter=',')
        
        fieldnames=['Gender','Birth Year', 'Year', 'Month', 'Number of Trips']
        
        csv_out.writerow(fieldnames)
    
        for row in people_all:
            csv_out.writerows(row)

In [196]:
import numpy as np
import pandas as pd

# Null values written as "/N" -- replace with nulls
peopledf=pd.read_csv("/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CB_real_metrics/PeopleGenderAge.csv", na_values=[r'\N'])

# Replace Gender w/ 'NA, M, F' vs. '0, 1, 2'
peopledf.Gender=peopledf.Gender.replace(0, 'U')
peopledf.Gender=peopledf.Gender.replace(1, 'M')
peopledf.Gender=peopledf.Gender.replace(2, 'F')

# Replace birth years < 1920 with nulls (see notebook "Identifying Outliers in Age Range" for reference)
peopledf.ix[peopledf['Birth Year'] < 1920, 'Birth Year'] = np.nan

# Drop rows with nulls (missing birth years and those < 1920)
peopledf=peopledf.dropna()
peopledf.head()

,Gender,Birth Year,Year,Month,Number of Trips
0,F,1944.0,2013,7,100
1,M,1921.0,2013,7,25
2,M,1987.0,2013,7,17294
3,F,1965.0,2013,7,2643
4,M,1946.0,2013,7,976


In [197]:
# Create bins for age ranges based off the birth year values
bins=[]
for number in range(1915, 2015, 10):
   bins.append(number)

# Add age ranges column to dataframe
peopledf['Age Ranges']=pd.cut(peopledf['Birth Year'], precision=0, bins=bins)
del peopledf['Birth Year']
peopledf.sort_values('Age Ranges')
peopledf.head()

,Gender,Year,Month,Number of Trips,Age Ranges
0,F,2013,7,100,"(1935, 1945]"
1,M,2013,7,25,"(1915, 1925]"
2,M,2013,7,17294,"(1985, 1995]"
3,F,2013,7,2643,"(1955, 1965]"
4,M,2013,7,976,"(1945, 1955]"


In [198]:
# Checking for nulls in age range column
x=peopledf['Age Ranges'].isnull()
np.sum(x)

0

In [199]:
# Change age ranges to intervals that will be easier to understand in a plot
unique_ranges=peopledf['Age Ranges'].unique()
from collections import defaultdict
x=defaultdict(str)

for line in unique_ranges:
    z=line[1:4]+'6'
    d=line[7:11]
    result=z+'-'+d
    x[line]=result

peopledf['Age Range']=peopledf['Age Ranges'].map(x)
peopledf.head()

,Gender,Year,Month,Number of Trips,Age Ranges,Age Range
0,F,2013,7,100,"(1935, 1945]",1936-1945
1,M,2013,7,25,"(1915, 1925]",1916-1925
2,M,2013,7,17294,"(1985, 1995]",1986-1995
3,F,2013,7,2643,"(1955, 1965]",1956-1965
4,M,2013,7,976,"(1945, 1955]",1946-1955


In [200]:
# Update dataframe to reflect new intervals
peopledf['Age Range']=peopledf['Age Ranges'].map(x)
peopledf.head()

,Gender,Year,Month,Number of Trips,Age Ranges,Age Range
0,F,2013,7,100,"(1935, 1945]",1936-1945
1,M,2013,7,25,"(1915, 1925]",1916-1925
2,M,2013,7,17294,"(1985, 1995]",1986-1995
3,F,2013,7,2643,"(1955, 1965]",1956-1965
4,M,2013,7,976,"(1945, 1955]",1946-1955


In [383]:
from bokeh.charts import Bar
from bokeh.models import NumeralTickFormatter
from bokeh.models import MultiSelect, CustomJS, ColumnDataSource
from bokeh.io import output_notebook, show, push_notebook, reset_output

reset_output()

p = Bar(peopledf, label='Age Range', values='Number of Trips', agg='sum', group='Gender',
        title="Number of Trips by Age Range, Grouped by Gender", legend='top_right')

x=peopledf['Age Range']
y=peopledf['Number of Trips']


# Create Widgets
from ipywidgets import interact, widgets

Months=widgets.SelectMultiple(
    options=['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
    value=['January'],
    description='Months',
    disabled=False)

Years=widgets.SelectMultiple(
    options=['2013', '2014', '2015', '2016'],
    value=['2013'],
    description='Years',
    disabled=False)

# Create function so plot is updated when specific selectors are picked
def update(Months, Years):
    months_list=[]
    years_list=[]
    if Months=="January": months_list.append(1)
    if Months=="February": months_list.append(2)
    if Months=="March": months_list.append(3)    
    if Months=="April": months_list.append(4)
    if Months=="May": months_list.append(5)
    if Months=="June": months_list.append(6)
    if Months=="July": months_list.append(7)
    if Months=="August": months_list.append(8)
    if Months=="September": months_list.append(9)
    if Months=="October": months_list.append(10)
    if Months=="November": months_list.append(11)
    if Months=="December": months_list.append(12)
    if Months==None: months_list.append(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
    if Years==2013: years_list.append(2013)
    if Years==2014: years_list.append(2014)
    if Years==2015: years_list.append(2015)
    if Years==2016: years_list.append(2016)  
    peopledf.loc[(peopledf['Month'].isin(months_list)) & peopledf['Year'].isin(years_list)]
    push_notebook()    

# Formatting the Graph
p.title.align='center'
p.title.text_font_size='14pt'

p.yaxis[0].formatter = NumeralTickFormatter(format="0,000,000")

p.xaxis.axis_label_standoff=20
p.yaxis.axis_label_standoff=20

p.xaxis.axis_label_text_font_size='13pt'
p.yaxis.axis_label_text_font_size='13pt'

p.xaxis.major_label_text_font_size='11.5pt'
p.xaxis.major_label_orientation=np.pi/4
p.yaxis.major_label_text_font_size='11.5pt'

p.yaxis.axis_label='Number of Trips'

In [384]:
output_notebook()

Loading BokehJS ...

In [396]:
show(p, notebook_handle=True)

In [386]:
interact(update, Months=SelectMultiple(
    options=['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
    value=['January'],
    description='Months',
    disabled=False), Years=SelectMultiple(
    options=['2013', '2014', '2015', '2016'],
    value=['2013'],
    description='Years',
    disabled=False))

<function __main__.update>

In [376]:
from ipywidgets import interact, SelectMultiple

In [381]:
Months=SelectMultiple(
    options=['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
    value=['January'],
    description='Months',
    disabled=False)

Months

In [49]:
# Plot in Bokeh
from bokeh.io import output_notebook, show, reset_output
from bokeh.layouts import widgetbox
from bokeh.models.widgets import MultiSelect
from ipywidgets import interact

reset_output()
output_notebook()

Month = MultiSelect(title="Months:", 
                           options=[("January", "January"), 
                                    ("February", "February"), 
                                    ("March", "March"),
                                    ("April", "April"),
                                    ("May", "May"),
                                    ("June", "June"),
                                    ("July", "July"),
                                    ("August", "August"),
                                    ("September", "September"),
                                    ("October", "October"),
                                    ("November", "November"),
                                    ("December", "December")])


Year = MultiSelect(title="Years:", 
                   options=[("2013", "2013"), 
                            ("2014", "2014"), 
                            ("2015", "2015"),
                            ("2016", "2016")])



show(widgetbox(Month))

show(widgetbox(Year))


Loading BokehJS ...

In [394]:
output_notebook()

Loading BokehJS ...

In [395]:
show(p)

In [415]:
import ipywidgets as widgets
from ipywidgets import interact

df=peopledf.copy()

items = ['All']+sorted(df['Year'].unique().tolist())
 
def view(x=''):
    if x=='All': return df
    return df[df['Year']==x]
 
w = widgets.Select(options=items)
interact(view, x=w)

<function __main__.view>

In [ ]:
from bokeh.charts import Bar
from bokeh.models import NumeralTickFormatter
from bokeh.io import output_notebook, show, push_notebook, reset_output
from ipywidgets import SelectMultiple, interact
reset_output()

p = Bar(peopledf, label='Age Range', values='Number of Trips', agg='sum', group='Gender',
        title="Number of Trips by Age Range, Grouped by Gender", legend='top_right')

x=peopledf['Age Range']
y=peopledf['Number of Trips']


# Create Widgets
from ipywidgets import interact, widgets

Months=widgets.SelectMultiple(
    options=['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
    value=['January'],
    description='Months',
    disabled=False)

Years=widgets.SelectMultiple(
    options=['2013', '2014', '2015', '2016'],
    value=['2013'],
    description='Years',
    disabled=False)

# Create function so plot is updated when specific selectors are picked
def update(Months, Years):
    months_list=[]
    years_list=[]
    if Months=="January": months_list.append(1)
    if Months=="February": months_list.append(2)
    if Months=="March": months_list.append(3)    
    if Months=="April": months_list.append(4)
    if Months=="May": months_list.append(5)
    if Months=="June": months_list.append(6)
    if Months=="July": months_list.append(7)
    if Months=="August": months_list.append(8)
    if Months=="September": months_list.append(9)
    if Months=="October": months_list.append(10)
    if Months=="November": months_list.append(11)
    if Months=="December": months_list.append(12)
    if Months==None: months_list.append(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
    if Years==2013: years_list.append(2013)
    if Years==2014: years_list.append(2014)
    if Years==2015: years_list.append(2015)
    if Years==2016: years_list.append(2016)  
    p.data_source.data=peopledf.loc[(peopledf['Month'].isin(months_list)) & peopledf['Year'].isin(years_list)]
    push_notebook()    

# Formatting the Graph
p.title.align='center'
p.title.text_font_size='14pt'

p.yaxis[0].formatter = NumeralTickFormatter(format="0,000,000")

p.xaxis.axis_label_standoff=20
p.yaxis.axis_label_standoff=20

p.xaxis.axis_label_text_font_size='13pt'
p.yaxis.axis_label_text_font_size='13pt'

p.xaxis.major_label_text_font_size='11.5pt'
p.xaxis.major_label_orientation=np.pi/4
p.yaxis.major_label_text_font_size='11.5pt'

p.yaxis.axis_label='Number of Trips'